In [89]:
import numpy as np
from math import tan, pi
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from matplotlib import cm 

In [103]:
# load tomography and iasp model
Pwave = np.loadtxt("PwaveSolution")
Swave = np.loadtxt("SwaveSolution")
iasp = np.loadtxt("iasp.txt")
T_ref = np.loadtxt("reference_temp.txt");
adiabat = np.loadtxt('/home/asaxena/opt/burnman-0.9.0/examples/adiabat_nmsz.txt') # adiabatic temp
# To = np.loadtxt('reference_temp.txt')
d = ( Pwave[:,3] <= 210.) & ( Pwave[:,3] >= 40. ) # last depth used in the current model
depth = Swave[d,3]  
delta_vs = Swave[d,2] ; delta_vp = Pwave[d,2]
x = (Swave[d,0] + 98)*111; # approximate conversions to the cartesian
y = (Swave[d,1] - 29)*111;

In [126]:
delta_vs[ delta_vs <= -4.5] = -4.5

In [127]:
# define mineral physics constants used in this study
data = np.array([('ol', 1, 4.2, 1.4, -0.017e9, -0.014e9, 129e9, 81e9, 3222, 0.201e-4, 0.139e-7, 0.1627e-2, 
                  1.658e2, 0.2332e-2, -0.3971e7) , 
       ('opx', 0., 7, 1.6, -0.027e9, -0.012e9, 109e9, 75e9, 3215, 0.3871e-4, 0.0446e-7, 0.03435e-2, 
       1.855e2, 0.233e-2, -0.6326e7) , 
       ('gt', 0.0, 4.4, 1.4, -0.019e9, -0.01e9, 171e9, 92e9, 3565, 0.0991e-4, 0.1165e-7, 1.0624e-2, 
       1.44e2, 0.188e-2, -0.135e7)], 
        dtype=[('name', 'U10'), ('conc', 'f4'), ('KPDer', 'f4'), ('MuPDer', 'f4'), 
        ('KTDer', 'f4'), ('MuTDer', 'f4'), ('K', 'f4'), ('Mu', 'f4'), ('rho', 'f4'), 
        ('a0', 'f4'), ('a1', 'f4'), ('a2', 'f4'), ('cp0', 'f4'), ('cp1', 'f4'), ('cp2', 'f4')])
A = 1.48e-1 ; H = 500e3; V = 20e-6; a = 0.15
R = 8.314

In [128]:
def density(a0, a1, a2, rho0, Ta):
    return (rho0 * np.exp (-a0 * (Ta - 300) - ( a1*(Ta - 300)**2 )/2 - a2*np.log (Ta - 300)))

In [129]:
def derivative (f, x, h, P, K, Kdash):
    return (f(x+h, P, K, Kdash) - f(x-h, P, K, Kdash)) / (2.0*h) # might want to return a small non-zero if ==0

def epsilon(x, P, K, Kdash):
    return  ( (1 - 2*x) ** (5/2) ) * (3 * K * x + ( ( 9 * K * (4 - Kdash) )/ 2 ) * (x ** 2) )  + P 

def solve_ep (f, x0, h, P, K, Kdash):
    lastX = x0
    nextX = lastX + 10 * h  # "different than lastX so loop starts OK
    #print(abs(lastX - nextX)) 
    while (abs(lastX - nextX) > h):  # this is how you terminate the loop - note use of abs()
        newY  = epsilon(nextX,  P, K, Kdash)                     # just for debug... see what happens
#         print (newY)     # print out progress... again just debug
        lastX = nextX
        nextX = lastX - newY / derivative(f, lastX, h, P, K, Kdash)  # update estimate using N-R
    return nextX

In [130]:
x0 = 1.e-8
pressures = adiabat[:,0]
depth_a =  adiabat[:,1] / 1e3 # tomography is in m, easier for comparison
T_a = adiabat[:,2]
strain = np.ones(len(pressures))
rho = np.ones(len(pressures))

# compositional dependence 
rho_avg = data['rho'][0] * data['conc'][0] + data['rho'][1] * data['conc'][1] +    \
data['conc'][2] * data['rho'][2]
K_avg = ( data['conc'][0] * data['K'][0] + data['conc'][1] * data['K'][1] + data['conc'][2]*data['K'][2])
Muavg = ( data['conc'][0] * data['Mu'][0] + data['conc'][1] * data['Mu'][1] + data['conc'][2]*data['Mu'][2])
KPDer_avg = ( data['conc'][0] * data['KPDer'][0] + data['conc'][1] * data['KPDer'][1] +   \
             data['conc'][2]*data['KPDer'][2])

for i in range (len(pressures)):
    Ta   = (T_a [ depth_a <= depth[i]].max()) # reference temperature with depth
    strain[i]     = solve_ep (epsilon, x0, 1.e-15, pressures[i], K_avg, KPDer_avg) # equation to get x
#     rho[i] = ( data['conc'][0] *  data['rho'][0] + data['conc'][1] * data['rho'][1] + \
#               data['conc'][2] * data['rho'][2] ) * ( ( 1 - ( 2 * strain[i] ) ) ** (3/2) )
    rho[i] = (( data['conc'][0] *  density ( data['a0'][0], data['a1'][0], data['a2'][0], data['rho'][0], Ta) ) 
    + ( data['conc'][1] *  density ( data['a0'][1], data['a1'][1], data['a2'][1], data['rho'][1], Ta) )
    + ( data['conc'][2] *  density ( data['a0'][2], data['a1'][2], data['a2'][2], data['rho'][2], Ta) ) ) \
    * ( ( 1 - ( 2 * strain[i] ) ) ** (3/2) )

In [131]:
print (rho)

[3127.61450605 3134.14482943 3140.63247648 3147.0757807  3153.47647475
 3159.84849001 3166.15265526 3172.41792681 3178.64494822 3184.83663904
 3191.01332466 3197.15330274 3203.25715817 3209.32546069 3215.35876543
 3221.35761347 3227.32253225 3233.25403613 3239.15262677 3245.0187936 ]


In [132]:
# corrections to the moduli
def moduli ( To, data, x, comp):
    return  ( data['KTDer'][comp] * (x - To) + (4./3) * data['MuTDer'][comp] * (x - To) )

# corrections to the moduli
def s_moduli ( To, data, x, comp):
    return  ( data['MuTDer'][comp] * (x - To) )

        
def moduli_der (To, data, x, comp):
    return ( data['KPDer'][comp] + (4/3) * data['MuPDer'][comp] ) *   \
( ( np.exp ( data['a0'][comp] * (x - 300) + ( data['a1'][comp] / 2 ) * (x - 300) ** 2  \
           ) ) * ( data['a0'][comp] + data['a1'][comp] * (x) )  * ( x - To ) ) 

# function for K and mu
def p_modulus ( To, data, x, comp , ep ):   
    return ( ( 1 - 2*ep )**(5/2) ) * ( moduli ( To, data, x, comp)   +   \
            ep * ( 5 * moduli ( To, data, x, comp)  - 3 * ( data['KTDer'][comp] ) *  \
          ( x- To ) * ( data['KPDer'][comp]  + (4/3) * data['MuPDer'][comp] ) - 3 * data['K'][comp] *   \
            ( moduli_der (To, data, x, comp ) ) ) ) 

# function for K and mu
def s_modulus ( To, data, x, comp , ep ):   
    return ( ( 1 - 2*ep )**(5/2) ) * ( s_moduli ( To, data, x, comp)   +   \
            ep * ( 5 * s_moduli ( To, data, x, comp)  - 3 * ( data['KTDer'][comp] ) *  \
          ( x- To ) * (  data['MuPDer'][comp] ) - 3 * data['K'][comp] * (3/4) * \
            ( moduli_der (To, data, x, comp ) ) ) )  

def density_der(comp, rho, To, data, x, ep):
    return - data['rho'][comp] * np.exp (- data['a0'][comp] * (x - 1300) - \
        ( data['a1'][comp] / 2) * (x - 1300) ** 2  ) * ( data['a0'][comp] + data['a1'][comp] * x ) * \
        ( x - To) * ( 1 - 2*ep )**(3/2.) 

In [133]:
# inversion for temperature
def double_derivative (f, x, h, delVp, ep, rho, To, data, P, Vso, Vpo):
    return (f(x+h, delVp, ep, rho, To, data, P, Vso, Vpo) + f(x-h, delVp, ep, rho, To, data, P, Vso, Vpo) -             2 * f(x-h, delVp, ep, rho, To, data, P, Vso, Vpo) ) / (h**2)  

def derivative (f, x, h, delVp, ep, rho, To, data, P, Vso, Vpo):
    return (f(x+h, delVp, ep, rho, To, data, P, Vso, Vpo) - f(x-h, delVp, ep, rho, To, data, P, Vso, Vpo)  ) / (2 * h)

def temperature_p ( x, delVp, ep, rho, To, data, P, Vso, Vpo):
    p_modulus0 = K_avg + (4/3)*Muavg     
    return ( 1/(2 * np.sqrt(rho_avg * p_modulus0) ) ) * ( data['conc'][0] *  p_modulus ( To, data, x, 0 , ep ) +
            data['conc'][1] *  p_modulus ( To, data, x, 1 , ep ) + data['conc'][2] *
            p_modulus ( To, data, x, 2 , ep ) ) - ( ( np.sqrt(p_modulus0)/( 2 * ( (rho_avg) ** 3/2 ) ) ) * \
            ( data['conc'][0] * density_der (0, rho, To, data, x , ep) +  data['conc'][1] * 
             density_der (1, rho, To, data, x, ep) + data['conc'][2] * density_der (2, rho, To, data, x, ep ) ) ) \
             + ( ( a * H * (4/3) * (Vso/Vpo)**2 ) * ( ( A * ( ( 2*np.pi )**a ) * 
             np.exp ( a * ( H + P*V ) / ( R * To ) ) ) ** -1 ) /(2 * R * ( To ** 2) * np.tan ( np.pi * a/2 ) ) ) \
             * (x - To) - delVp

def temperature_s ( x, delVs, ep, rho, To, data, P, Vso, Vpo):
    s_modulus0 = Muavg  
    data['KPDer'][:] = 0 ;
    return ( 1/(2 * np.sqrt(rho_avg * s_modulus0) ) ) * (data['conc'][0] * ( s_modulus ( To, data, x, 0 , ep ) +
            data['conc'][1] *  s_modulus ( To, data, x, 1 , ep ) + data['conc'][2] *
            s_modulus ( To, data, x, 2 , ep ) ) ) - ( ( np.sqrt(s_modulus0)/( 2 * ( (rho_avg) ** 3/2 ) ) ) * \
            ( data['conc'][0] * density_der (0, rho, To, data, x , ep) +  data['conc'][1] * 
             density_der (1, rho, To, data, x, ep) + data['conc'][2] * density_der (2, rho, To, data, x, ep ) ) ) \
             + ( ( a * H ) * ( ( A * ( ( 2*np.pi )**a ) * 
             np.exp ( a * ( H + P*V ) / ( R * To ) ) ) ** -1 ) /(2 * R * ( To ** 2) * np.tan ( np.pi * a/2 ) ) ) \
             * (x - To) - delVs


def solve_T (f, x0, h, delVp, ep, rho, To, data, P, Vso, Vpo):
    lastX = x0
    nextX = lastX +  10 * h  # "different than lastX so loop starts OK
    while (abs(lastX - nextX) > h):  # this is how you terminate the loop - note use of abs()        
        newY  = f (nextX, delVp, ep, rho, To, data, P, Vso, Vpo)          # just for debug... see what happens
#         print (newY)     # print out progress... again just debug
        lastX = nextX
        nextX = lastX -  f (nextX,  delVp, ep, rho, To, data, P, Vso, Vpo) / double_derivative(f, lastX, h,  delVp, ep, rho, To, data, P, Vso, Vpo)  # update estimate using N-R
    return nextX

In [134]:
Vp_o = np.ones(len(depth)) ; Vs_o = np.ones(len(depth))
dVp = np.ones(len(depth)) ; Vp = np.ones(len(depth))
To = np.ones(len(depth)) ; ep_cal = np.ones(len(depth))
rho_cal = np.ones(len(depth)); T_p = np.ones(len(depth)) 
P = np.ones(len(depth)) ; dVs = np.ones(len(depth))
T_s = np.ones(len(depth))

In [135]:
# invert for temperatures
x0 = 200;
depth_a =  adiabat[:,1] / 1e3 # tomography is in m, easier for comparison
T_a = adiabat[:,2]

for i in range (len(depth)):
    data['a2'][:] = 0; # ignore second terms
    Vp_o[i]  = (iasp[:,1][iasp[:,0] <= depth[i]].max())*1.e3 # find P wave reference velocity for the tomography
    Vs_o[i]  = (iasp[:,2][iasp[:,0] <= depth[i]].max())*1.e3 # find S wave reference velocity for the tomography
    To[i]   = (T_ref[:,1][T_ref[:,0] <= depth[i]].max()) + 273# reference temperature with depth
    dVp[i]   = delta_vp[i] * Vp_o[i]/100; 
    dVs[i]   = delta_vs[i] * Vs_o[i]/100; 
    Vp[i]    = dVp[i] + Vp_o[i]
    ep_cal[i]  = strain [depth_a <= depth[i]].max()
    P[i]     = (pressures[depth_a <= depth[i]].max())
    rho_cal[i] = rho [depth_a <= depth[i]].max()
    # equation to get x
#     T_p[i]     = solve_T (temperature_p, x0, 1,  dVp[i], ep_cal[i], rho_cal[i], To[i], data, P[i], 
#                          Vs_o[i], Vp_o[i])
    T_s[i]     = solve_T (temperature_s, x0, 1,  dVs[i], ep_cal[i], rho_cal[i], To[i], data, P[i], 
                         Vs_o[i], Vp_o[i])


In [136]:
# print (max(T_p-To) , min(T_p-To))
print (max(T_s-To), min(T_s-To))

464.09846849059795 -516.3606103502932


In [55]:
print (rho_avg)

3227.81


In [59]:
d_cal   = np.reshape(depth,(len(d_q), npts, npts))
T_cal   = np.reshape(T_results, (len(d_q), npts, npts))
dVp_cal = np.reshape(dVp, (len(d_q), npts, npts))
Vp_cal = np.reshape(Vp, (len(d_q), npts, npts))
lat_cal = np.reshape(latitude,(len(d_q), npts, npts))
long_cal= np.reshape(longitde,(len(d_q), npts, npts))
n = 0
(fig, (ax1, ax2)) = plt.subplots(1, 2, figsize=(20,6))
plt.rcParams.update({'font.size': 14})
CB = ax1.pcolormesh(long_cal[n,:, :], lat_cal[n, :, :], T_cal[n ,:, :], cmap=cm.hot, shading='gouraud')
CB1 = ax2.pcolormesh(long_cal[n,:, :], lat_cal[n, :, :], Vp_cal[n ,:, :], cmap=cm.hot, shading='gouraud')
cbar = plt.colorbar(CB, shrink=0.6, ax=ax1)
cbar2 = plt.colorbar(CB1, shrink=0.6, ax=ax2)

CB.axes.set_xlim([-92, -74])
CB.axes.set_ylim([27, 41])
CB1.axes.set_xlim([-92, -74])
CB1.axes.set_ylim([27, 41])
plt.show()
print(np.unique(longitde)[n])

NameError: name 'd_q' is not defined

In [13]:
# addting a buffer layer
longitde_c = np.concatenate([np.reshape (np.tile(grid_x, (2,1)), (npts*npts*2)), longitde])
latitude_c   = np.concatenate([np.reshape (np.tile(grid_y, (2,1)), (npts*npts*2)), latitude])
depth_c    = np.concatenate([5*np.ones(npts*npts), 40*np.ones(npts*npts), depth])
T_c = np.concatenate([423*np.ones(npts*npts), 773*np.ones(npts*npts), T_results])
print (np.shape(latitude_c), np.shape(depth))

r = (6370 - depth_c)*1e3
longitude = (360 + longitde_c)*np.pi/180
latitde = (90 - latitude_c)*np.pi/180
ind = np.lexsort((r, longitude, latitde))
np.savetxt('initial_temperature_Pder_no_root.txt', \
            np.stack((r[ind], longitude[ind], latitde[ind], T_c[ind]), axis=-1))

(382500,) (337500,)


In [59]:
print (d_q[2
          ])

130.0
